## Ground Truth dataset 

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

import time
import os
import json
import re

In [3]:
faqs_url = 'https://philnat.unibas.ch/de/faqs/'
bot_name = 'Unibasel_RAG_bot/1.0'

In [4]:
def count_characters_on_webpage(url):
 
    headers = {'User-Agent': bot_name}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all text from the webpage
        text = soup.get_text()
        
        # Count the characters
        character_count = len(text)
        print(f"The URL contains {character_count} characters.")
        
        return character_count
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None

In [5]:
count_characters_on_webpage(faqs_url)

The URL contains 12786 characters.


12786

In [6]:
def extract_text_from_webpage(url):
 
    headers = {'User-Agent': bot_name}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all text from the webpage
        text = soup.get_text()
    
        
        return text
    else:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None
        

In [6]:
text = extract_text_from_webpage(faqs_url)

In [7]:
text

'\n\n\n\n\nFAQs | Philosophisch-Naturwissenschaftliche \r\nFakultät | Universität Basel\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\nDiese Webseite benötigt JavaScript, um richtig zu funktionieren.\nWie aktiviere ich JavaScript in diesem Browser?\n\n\n\n\n\n\n\nSuche\n\n\n\n\n\n\n\n\nSuche\n\n\nSuche\n\n\n\n\n\n\n\n\n\n\n\nPhilosophisch-Naturwissenschaftliche \r\nFakultät\n\n\n\n\n\n\n\n\n\nSuche\n\n\n\n\nSuche\n\n\nSuche\n\n\n\n\n\n\n\n\n\n\n                                Fakultät\n                            \n\n\n\n                                Studium\n                            \n\n\n\n                                Forschung\n                            \n\n\n\n                                Termine & Aktuelles\n                            \n\n\n\n\n\n\n\n\n\n\n\n\nDekanat\n\n\nKommissionen\n\n\nProfessoren\n\n\nGeschichte\n\n\n\n\nBachelorstudiengänge\n\n\nMasterstudiengänge\n\n\nausserfakultäre Studienfächer\n\n\nExamen\n\n\nAbschlüsse\n\n\nStudiendekanat\n\n\nBa

In [7]:
# module for regular expressions
import re


def extract_faqs(text):

    pattern_1 = r'(Studium allgemein\n.*?\?)(.*?)(?=\n\s*ANTWORT EINBLENDEN)'
    #  r'(.*?)\?([^?]+)(?=ANTWORT EINBLENDEN|$)'
    pattern_2 = r'(\n.*?\?)(.*?)(?=\n\s*ANTWORT EINBLENDEN)'
    
    # finds all matches
    matches_1 = re.findall(pattern_1, text, re.DOTALL)
    matches_2 = re.findall(pattern_2, text, re.DOTALL)

    # except the first (pattern_1)
    folders = ["Prüfungen zu Hauptvorlesungen", "Prüfungen zu Lehrveranstaltungen", "Noten und Bewertungen", "Studienabschluss"] 

    cleaned_faqs = [{'question': q.replace("Studium allgemein", "").strip(), 'answer': a.replace("-\xa0\xa0\xa0\xa0", "").strip()} for q, a in matches_1]

    for q, a in matches_2:
        q_exl = []
        for f in folders:
            
            if q.replace("ANTWORT EINBLENDEN\n    \n        FRAGE EINBLENDEN\n", "").strip().startswith(f):
                cleaned_faqs.append({'question': q.replace("ANTWORT EINBLENDEN\n    \n        FRAGE EINBLENDEN\n", "").strip().replace(f, "").strip(), 'answer': a.strip()})
                q_exl.append(q)
                
        if  not q in q_exl:
            cleaned_faqs.append({'question': q.replace("ANTWORT EINBLENDEN\n    \n        FRAGE EINBLENDEN\n", "").strip(), 'answer': a.strip()})

    cleaned_faqs = [x for x in cleaned_faqs if not x['question'].startswith("FAQs")]


    return cleaned_faqs
    

In [180]:
cleaned_docs = extract_faqs(text)
# 6 + 3 + 2 + 2 + 5
len(cleaned_docs)

16

In [181]:
# Actually, we have 6 + 3 + 2 + 2 + 5 = 18 
# the problem is that they aren’t questions... So I decided to reformulate them into questions and add them manually
# 'Bachelorfeier' ->  Wann und für wen findet die Bachelorfeier?
# 'Ich habe vergessen, eine Lehrveranstaltung zu belegen und die Belegfrist ist abgelaufen.' -> Welche Konsequenzen könnte es haben, wenn man vergisst, eine Lehrveranstaltung zu belegen und die Belegfrist verstreichen lässt?
add_docs = [{'question': 'Welche Konsequenzen könnte es haben, wenn man vergisst, eine Lehrveranstaltung zu belegen und die Belegfrist verstreichen lässt?', 'answer': 'Es ist obligatorisch, jede Lehrveranstaltung zu belegen. Wurde nicht belegt, können die KP für diese Veranstaltung nicht gutgeschrieben werden, auch wenn die Prüfung / Übung / Exkursion etc. bestanden wurde. Sie können per Email an studiendekanat-philnat@clutterunibas.ch einen Antrag auf nachträgliche Belegung stellen. Falls diesem Antrag vom Studiendekan stattgegeben wird, kostet die Nachbelegung in der Regel CHF 100.'},
{'question': 'Wann und für wen findet die Bachelorfeier?', 'answer': 'Jedes Jahr im Herbst findet die Bachelorfeier der Phil.-Nat. Fakultät statt. Alle Absolventinnen und Absolventen des jeweiligen Jahres (HS und FS) erhalten zu diesem festlichen Anlass eine Vorinformation sowie eine Einladung.' }]


cleaned_docs = cleaned_docs + add_docs 
len(cleaned_docs)

18

In [182]:
cleaned_docs

[{'question': 'Was sind die rechtlichen Grundlagen meines Studiums?',
  'answer': 'Alle generellen Fragen rund um das Studium an der Universität Basel werden in den folgenden Ordnungen geregelt: Studierendenordnung (gilt für alle Studierenden der Uni Basel): https://www.unibas.ch/de/Studium/Studierendenordnung.html Bachelorstudium: Ordnung der Philosophisch Naturwissenschaftlichen Fakultät der Universität Basel für das Bachelorstudium-\xa0\xa0 \xa0 Masterstudium: Ordnung der Philosophisch-Naturwissenschaftlichen Fakultät der Universität Basel für das MasterstudiumDiese Ordnungen gelten für alle Studierenden, \xa0die entweder an der Phil.-Nat. Fakultät immatrikuliert sind, oder eine Lehrveranstaltung besuchen, die von der Phil.-Nat. Fakultät angeboten wird): https://philnat.unibas.ch/de/studium/'},
 {'question': 'Was ist eine Datenabschrift?',
  'answer': 'Eine Datenabschrift ist die offizielle Bescheinigung Ihrer an der Universität Basel absolvierten Studienleistungen mit dem Ausweis d

In [326]:
# filename = 'FAQs.json'
# with open(filename, 'w') as f:
#     json.dump(cleaned_docs, f, indent=4)

In [19]:
faqs_url_2 = 'https://www.unibas.ch/de/Studium/FAQ-Studium.html'
text_2 = extract_text_from_webpage(faqs_url_2)
import re

def extract_qa(text):
    # Pattern to match questions and answers
    pattern = r'(\n\n[^\n]+\?\n\n\n\n)([^?]+?)(?=\n\n[^\n]+\?\n\n|\Z)'
    
    # Find all matches
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Process matches to clean up the text
    qa_pairs = []
    for match in matches:
        question = match[0].strip()
        answer = match[1].strip()
        
        # Remove any extra newlines and spaces
        question = re.sub(r'\s+', ' ', question)
        answer = re.sub(r'\s+', ' ', answer)
        
        qa_pairs.append((question, answer.replace('Antwort einblenden Frage einblenden', '')))
    
    return qa_pairs

extract_qa(text_2)

[('Was kann ich an der Universität Basel alles studieren?',
  'Die Website zum Studienangebot gibt Ihnen einen Überblick, was Sie an der Universität Basel auf Bachelor- und Masterstufe studieren können. '),
 ('Wie viele Fächer kann ich gleichzeitig studieren?',
  'Das hängt von der gewählten Fakultät (und Ihren Zeitressourcen) ab: In Theologie, Rechtswissenschaft, Medizin, Psychologie, Wirtschaftswissenschaften und in der Regel in den Naturwissenschaften gibt es jeweils einen geschlossenen Studiengang mit Teilfächern und Schwerpunkten sowie einen zusätzlichen Wahlbereich. In den Geistes- und Sozialwissenschaften werden in der Regel zwei Bachelor- und Masterstudienfächer miteinander kombiniert; hinzu kommt ein Wahlbereich, in dem frei belegt werden kann. Die genauen Bestimmungen zur Fächerkombination finden Sie unter www.unibas.ch/anmeldung Die gleichzeitige Einschreibung in mehreren Studiengängen ist in der Regel nicht möglich. Über Ausnahmen entscheidet das Rektorat.  Ich schwanke zwi

In [17]:
# must be 12 
# again, because 2 of them are not questions :/ 
len(extract_qa(text_2))

10

*I didn't add the last one to the test dataset. These questions will not help test the system we want to build; it is more helpful to have this as a source.*

## Update FAQs using .docx file

In [3]:
filename = 'FAQs.json'

with open(filename, 'r') as file:
    faqs_data = json.load(file)

print("JSON file imported successfully.")

JSON file imported successfully.


In [4]:
import docx

In [5]:
doc_file = "faqs_new.docx"

def extract_qa(file_path):
    doc = docx.Document(file_path)
    qa_pairs = []
    current_question = None
    current_answer = None

    for paragraph in doc.paragraphs:
        text = paragraph.text.strip()
        if text.startswith("Frage:"):
            if current_question and current_answer:
                qa_pairs.append((current_question, current_answer))
            current_question = text[6:].strip()
            current_answer = None
        elif text.startswith("Antwort:"):
            current_answer = text[8:].strip()
    
    if current_question and current_answer:
        qa_pairs.append((current_question, current_answer))
    
    return qa_pairs

        
    
qa_pairs = extract_qa(doc_file)

In [6]:
qa_pairs[0][0]

'Wie melde ich mich zu den Examen an?'

In [7]:
len(qa_pairs)

16

In [8]:
def create_dict(qa_pairs):
    data_full = []
    
    for idx in range(len(qa_pairs)):
        data = {}
        data["question"] = qa_pairs[idx][0]
        data["answer"] = qa_pairs[idx][1]
        data_full.append(data)

    return data_full

qa_pairs_dict = create_dict(qa_pairs)

qa_pairs_dict

[{'question': 'Wie melde ich mich zu den Examen an?',
  'answer': 'Die Anmeldung zu Examen von Hauptvorlesungen der Phil.-Nat. Fakultät erfolgt online in den Services. Eine Anleitung dazu finden Sie auf https://philnat.unibas.ch/de/examen unter "Informationen & Downloads". Die Anmeldung kann nur während der Anmeldefrist durchgeführt werden, welche auch auf unserer Examenswebseite bekanntgegeben wird unter "Fristen". Ausserhalb der Anmeldefrist ist keine Wiederholung möglich.'},
 {'question': 'Muss ich die Hauptvorlesung erneut belegen, um mich zum Wiederholversuch anzumelden?',
  'answer': 'Nein, die Anmeldung zum Wiederholversuch erfolgt online in den Services auf der Belegung, auf welcher der Erstversuch nicht bestanden wurde. Eine erneute Belegung ist nur notwendig, um einen ADAM Zugang zu den Vorlesungsmaterialien zu erhalten.'},
 {'question': 'Ich möchte das Examen noch nicht machen, kann ich es hinausschieben? Wenn ja wie lange?',
  'answer': 'Es gibt keine Frist, innerhalb welch

In [9]:
faqs_data_new = faqs_data + qa_pairs_dict
len(faqs_data_new)

34

In [10]:
# filename = 'FAQs_updated.json'
# with open(filename, 'w') as f:
#     json.dump(faqs_data_new, f, indent=4)

## Separate answers and links

It is necessary to separate links and text to compute more precise scores. 


In [13]:
def extract_replace_urls(text):
    # Regular expression for URLs
    url_pattern = r'http[s]?://[^\s]+'  # Matches http or https URLs

    links = re.findall(url_pattern, text)
    
    replaced_text = re.sub(url_pattern, ' ', text)
    return links, replaced_text

text = faqs_data_new[0]['answer']

links, cleaned_text = extract_replace_urls(text) 

print("Original text: ", text)

print("--------------------------------------------------")

print("Links: ", links)

print("Cleaned text: ", cleaned_text)

Original text:  Alle generellen Fragen rund um das Studium an der Universität Basel werden in den folgenden Ordnungen geregelt: Studierendenordnung (gilt für alle Studierenden der Uni Basel): https://www.unibas.ch/de/Studium/Studierendenordnung.html Bachelorstudium: Ordnung der Philosophisch Naturwissenschaftlichen Fakultät der Universität Basel für das Bachelorstudium-     Masterstudium: Ordnung der Philosophisch-Naturwissenschaftlichen Fakultät der Universität Basel für das MasterstudiumDiese Ordnungen gelten für alle Studierenden,  die entweder an der Phil.-Nat. Fakultät immatrikuliert sind, oder eine Lehrveranstaltung besuchen, die von der Phil.-Nat. Fakultät angeboten wird): https://philnat.unibas.ch/de/studium/
--------------------------------------------------
Links:  ['https://www.unibas.ch/de/Studium/Studierendenordnung.html', 'https://philnat.unibas.ch/de/studium/']
Cleaned text:  Alle generellen Fragen rund um das Studium an der Universität Basel werden in den folgenden Ordn

In [15]:
faqs_data_cleaned = []

for i in range(len(faqs_data_new)):
    text = faqs_data_new[i]['answer']
    links, cleaned_text = extract_replace_urls(text) 

    record = {
        "question": faqs_data_new[i]['question'],
        "answer": cleaned_text,
        "links": links
    }

    faqs_data_cleaned.append(record)
    

In [16]:
filename = 'FAQs_cleaned.json'
with open(filename, 'w') as f:
    json.dump(faqs_data_cleaned, f, indent=4)

---
* Author: Anastasiia Popova
* Email: anastasiia.popova@stud.unibas.ch

[Perplexity AI](https://www.perplexity.ai/) assisted in code writing, editing, and more effective information searches. The generated output underwent critical evaluation. The author is solely responsible for the content.
